In [ ]:
# Convet labeled data in JARVIS to the Lightning Pose format
# 1. Select N cameras 
# 2. Fuse synchronized camera views into one frame 
# 3. Create new keypoints (keypoint + camera view)
#
# Lightening Pose provides a function to convert DLC labeled data to LP labeled data
# Reference
# https://github.com/danbider/lightning-pose/blob/main/scripts/converters/dlc2lp.py



In [14]:
import glob
import os
import shutil

import numpy as np
import pandas as pd
from PIL import Image
import cv2

In [2]:
dataset_name = '6cam_dataset_231216'
jarvis_dir = r'E:\Hand_tracking\Datasets\annotation' # path to the annotation folder of JARVIS project
lp_dir = r'E:\Hand_tracking\LP_projects\LP_240129' # path to the conversion folder of lp project
cameras = ["camTo", "camBL"]
frame_rate = 100
# Define how camera views are fused
frame_size = [960, 960] # width and hight in pixels
# Fuse two frames side-by-side
x_orgs = [0, frame_size[0]] # origin in the new x axis
y_orgs = [0, 0] # origin in the new y axis

# Find all labeled data in JARVIS project
prefix_year = '2023' # Trial names starts with XXXX year. 
dirs = [filename for filename in os.listdir(os.path.join(jarvis_dir, dataset_name)) if filename.startswith(prefix_year)]
dirs.sort()
dfs = []


In [3]:
# Create a new video name (date + camera_0 + camera_1 + ...)
for i,c in enumerate(cameras):
    if i == 0:
        new_c = c
    else:
        new_c = new_c + '_' + c

In [9]:
for d in dirs:
    vid = d + '_' + new_c
    dfs_trial = []
    for i, c in enumerate(cameras):
        csv_file = glob.glob(os.path.join(jarvis_dir, dataset_name, d, c, "annotations.csv"))[0]

        df1 = pd.read_csv(csv_file, on_bad_lines='skip', header = None, index_col=0) 
        df2 = pd.read_csv(csv_file, skiprows=4, header = None, index_col=0) 
        last_column = df2.shape[1]
        df2 = df2.drop(columns=[last_column],axis='columns')

        # Remove entities row
        isNotEntities = [x != 'entities' for x in df1.index.values]
        df1 = df1.iloc[isNotEntities]

        # Rename bodypart names (bodypart + camera view)
        isBodyparts = [x == 'bodyparts' for x in df1.index.values]
        df1.iloc[isBodyparts] = [x + '_' + c for x in df1.iloc[isBodyparts].values]

        # Find coords row and remove state columns
        isCoords = [x == 'coords' for x in df1.index.values]
        isXY = [s != 'state' for s in df1.iloc[isCoords].values]
        df1 = df1.iloc[:, isXY[0]]
        df2 = df2.iloc[:, isXY[0]]

        # Shift origin 
        isCoords = [x == 'coords' for x in df1.index.values]
        isX = [x == 'x' for x in df1.iloc[isCoords].values]
        df2.iloc[:, isX[0]] = [x + x_orgs[i] for x in df2.iloc[:, isX[0]].values]
        isY = [x == 'y' for x in df1.iloc[isCoords].values]
        df2.iloc[:, isY[0]] = [x + y_orgs[i] for x in df2.iloc[:, isY[0]].values]

        # Replace image file name with its file path
        imgs = list(df2.index.values)
        # Change .jpg to .png (JARVIS- .jpg, LP/DLC- .png)
        im_idx = [i[6:len(i)-4] for i in imgs]
        imgs_new =['img' + format(int(i), '04d') + ".png" for i in im_idx]
        new_col = [f"labeled-data/{vid}/{i}" for i in imgs_new]
        df2.index = new_col

        df_tmp = pd.concat([df1,df2])

        os.makedirs(os.path.join(lp_dir, "Jarvis2LP", dataset_name, d, c), exist_ok=True)
        df_tmp.to_csv(os.path.join(lp_dir, "Jarvis2LP", dataset_name, d, c, "CollectedData.csv"), header = False)
        df = pd.read_csv(os.path.join(lp_dir, "Jarvis2LP" ,dataset_name, d, c, "CollectedData.csv"), header = [0,1,2], index_col=0)
        dfs_trial.append(df)
    df_trial = pd.concat(dfs_trial, axis=1) 
    dfs.append(df_trial)
    
df_all = pd.concat(dfs)

# save concatenated labels 
df_all.to_csv(os.path.join(lp_dir, "Jarvis2LP", dataset_name, "CollectedData.csv"))
            
    

In [19]:
# Create fused videos by combining frames side by side
src_img_dir = r'E:\Hand_tracking\Recordings\Images'
os.makedirs(os.path.join(lp_dir,'videos'), exist_ok=True)

for d in dirs:
    session = d[0:10]
    # Get the folder for videos
    image_path_trial = os.path.join(src_img_dir, session, d)
    # Load the images
    image_folder1 = os.path.join(image_path_trial, 'camTo')
    image_folder2 = os.path.join(image_path_trial, 'camBL')

    image_names1 = [img for img in os.listdir(image_folder1) if img.endswith(".bmp")]
    image_names2 = [img for img in os.listdir(image_folder2) if img.endswith(".bmp")]

    # Get the image size 
    frame = cv2.imread(os.path.join(image_folder1, image_names1[0]))
    height, width, layers = frame.shape

    # New video name
    vid = d + '_' + new_c
    
    # Initiate a video for combined images    
    video_save_path = os.path.join(lp_dir, "videos", vid+'.mp4')
    video = cv2.VideoWriter(video_save_path,  
                    fourcc = cv2.VideoWriter_fourcc(*'mp4v'), 
                    fps=frame_rate,                                       
                    frameSize=(width*2, height)) # combine them horizontally: width*2

    for index, (image_name1, image_name2) in enumerate(zip(image_names1, image_names2)): 
        image1 = cv2.imread(os.path.join(image_folder1, image_name1))
        image2 = cv2.imread(os.path.join(image_folder2, image_name2))

        # Concatenate images horizontally
        combined_image = np.concatenate((image1, image2), axis=1)

        # Write the combined image to the video
        video.write(combined_image)

    cv2.destroyAllWindows()    
    video.release()

In [31]:
# Fuse labeled-frames side by side
for d in dirs:
    # New video name
    vid = d + '_' + new_c
    # Create a folder to save fused images in each video (trial)
    os.makedirs(os.path.join(lp_dir,"labeled-data", vid), exist_ok = True)
    # Load the images
    image_folder1 = os.path.join(jarvis_dir, dataset_name, d, 'camTo')
    image_folder2 = os.path.join(jarvis_dir, dataset_name, d, 'camBL')

    image_names1 = [img for img in os.listdir(image_folder1) if img.endswith(".jpg")]
    image_names2 = [img for img in os.listdir(image_folder2) if img.endswith(".jpg")]

    # Get the image size 
    frame = cv2.imread(os.path.join(image_folder1, image_names1[0]))
    height, width, layers = frame.shape
    
    for index, (image_name1, image_name2) in enumerate(zip(image_names1, image_names2)): 
        image1 = Image.open(os.path.join(image_folder1, image_name1))
        image2 = Image.open(os.path.join(image_folder2, image_name2))

        # Concatenate images horizontally
        fused_image = np.concatenate((image1, image2), axis=1)
        
        # Save fused image
        im_idx = image_name1[6:len(image_name1)-4] 
        new_name ='img' + format(int(im_idx), '04d') + ".png" 
        cv2.imwrite(os.path.join(lp_dir,"labeled-data", vid, new_name), fused_image)

In [32]:
# check
for im in df_all.index:
    assert os.path.exists(os.path.join(lp_dir, im))

In [ ]:
# In JARVIS, we can create multiple labeled datasets. 
# To concatenate them for a Lightning Pose project, we first convert each dataset and save them in a conversion folder.
# We then concatenate "CollectedData.csv" of each dataset and save it in the main dirrectory of the LP project.  
datasets = os.listdir(os.path.join(lp_dir, "Jarvis2LP"))
dfs = []
for d in datasets:
    df = pd.read_csv(os.path.join(lp_dir, "Jarvis2LP", d, "CollectedData.csv"), header = [0,1,2], index_col=0)
    dfs.append(df)
    
df_all = pd.concat(dfs)
# save concatenated labels 
df_all.to_csv(os.path.join(lp_dir,  "CollectedData.csv"))

In [ ]:
# Check if all labled frames were copied to the labeled-data folder
for im in df_all.index:
    assert os.path.exists(os.path.join(lp_dir, im))

In [ ]:
# Check if some frames are labeled more than once when we combine multiple labeled datasets, .
len(df_all.index) == len(set(df_all.index))